<center>

![huawei-logo](https://www-file.huawei.com/-/media/corporate/images/home/logo/huawei_logo.png)

<p><b>Летняя школа. "Современные методы теории информации, оптимизации и управления".</p></b>
Sirius-2020.
<center> <b>Авторы материала: к.т.н. Антонов Лев, Власов Роман.

In [1]:
!pip install git+https://github.com/keras-team/keras-tuner.git@1.0.2rc1  &> /dev/null
!pip install autokeras  &> /dev/null


# Практическое знакомство с AutoKeras
![autokeras-logo](https://camo.githubusercontent.com/5eedb6a5c3303767497912731e006b662bf83490/68747470733a2f2f6175746f6b657261732e636f6d2f696d672f726f775f7265642e737667)

Одной из самых новых и мощных концепций на сегодняшний день является направление ___"Поиск нейросетевых архитектур" (Neural Architecture Search (NAS))___. NAS - это, по сути, метод устранения ограничений человечских знаний и эвристик при ручном построении архитектур нейронных сетей.

Чтобы успешно использовать NAS в прошлом, требовались очень сложные реализации сценариев Tensorflow, PyTorch или Keras. Помимо этого, рассчитать требования к оборудованию на уровне предприятия. Для упрощения задачи поиска команда разработчиков из "Texas A & M Lab" разработала платформу с открытым исходным кодом, созданную с помощью Keras, чтобы предоставить возможность использовать NAS любому заядлому пользователю Keras + python. 
Версия 1.0 платформы **AutoKeras** была выпущена только в январе 2019 года после года, предшествующего предварительным версиям, что позволило ей выйти без большого количества багов.

В библиотеке используются **самые современные алгоритмы NAS**.

AutoKeras поддерживает несколько стандартных конструкций, обладающих чрезвычайно простым интерфейсом, для решения ряда распространенных задач:
* Классификация изображений (**ImageClassifier**).
* Регрессия изображения (**ImageRegressor**).
* Классификация текста (**TextClassifier**).
* Регрессия текста (**TextRegressor**).
* Классификация структурированных данных (**StructuredDataClassifier**).
* Регрессия структурированных данных (**StructuredDataRegressor**).

Также есть возможность настроить собственную модель в ручном режиме.
Для этого в AutoKeras предусмотрен класс **AutoModel**.
Здесь присутствует возможность тонкой настройки алгоритма и указание списка конкретных блоков, на которые алгоритм должен обратить внимание в первую очередь при построении моделей.


**Nodes** (Список возможных входных блоков):
* ImageInput
* Input
* StructuredDataInput
* TextInput

**Blocks** (Список блоков - потенциальных скрытых слоев модели) :
* ImageAugmentation
* Normalization
* TextToIntSequence
* TextToNgramVector
* CategoricalToNumerical
* ConvBlock
* DenseBlock
* Embedding
* Merge
* ResNetBlock
* RNNBlock
* SpatialReduction
* TemporalReduction
* XceptionBlock
* ImageBlock
* StructuredDataBlock
* TextBlock
* ClassificationHead
* RegressionHead

# Image Classification

Первый шаг - подготовка данных. Здесь мы используем набор данных MNIST.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.python.keras.utils.data_utils import Sequence
import autokeras as ak
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)  # (60000, 28, 28)
print(y_train.shape)  # (60000,)
print(y_train[:3])  # array([7, 2, 1], dtype=uint8)

11493376/11490434 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
[5 0 4]


Указываем путь, по которому будут храниться история и логи поиска

In [ ]:
OUTPUT_PATH = "0_0_AutoMLArchSearchClassiferMNIST/"

**Разберем параметры класса**

``` python

autokeras.ImageClassifier(
    num_classes=None, # По умолчанию None. Если None, это будет выведено из данных.
    multi_label=False,
    loss=None,        # функция потери Кераса. По умолчанию используется «binary_crossentropy» 
                      # или «categoryorical_crossentropy» в зависимости от количества классов.
    
    metrics=None,     # список метрик Кераса. По умолчанию используется «точность».
    project_name="image_classifier",
    max_trials=100,   # Максимальное количество различных моделей Keras,которые можно попробовать. 
                      # Поиск может завершиться до достижения max_trials. По умолчанию 100. 
    directory=None,
    objective="val_loss", # Имя метрики модели для минимизации или максимизации, например, «val_accuracy». 
                          # По умолчанию «val_loss».
    tuner=None,       # строка или подкласс модуля AutoTuner. 
                      # Если строка, то значения должны быть следующие 'greedy', 'bayesian', 'hyperband' or 'random'.
                      # По сути это выбор стратегии поиска архитектуры, выбор движка.
                      # Если значение не указано, то алгоритм основываясь на понимании контекста задачи сам выбирает 
                      # стратегию поиска
    overwrite=False,
    seed=None,
    **kwargs
) 
``` 

**Рассмотрим также параметры функции fit**

``` python
ImageClassifier.fit(
    x=None,          # Union[numpy.ndarray, tensorflow.data.Dataset, None]
                     # numpy.ndarray или tensorflow.Dataset.

    y=None,          # Union[numpy.ndarray, tensorflow.data.Dataset, None]
                     # numpy.ndarray или tensorflow.Dataset.

    epochs=None,     # Количество эпох для обучения каждой модели при
                     # поиске. Если не указано, по умолчанию максимально 
                     # модель обучается 1000 эпох, но мы прекращаем 
                     # обучение, если ошибка на валидации перестает
                     # улучшаться в течение 10 эпох (если только вы не 
                     # указан параметр callbacks = EarlyStopping, в этом 
                     # случае EarlyStopping будет определить раннюю 
                     # остановку).

    callbacks=None,  # Optional[List[tensorflow.keras.callbacks.Callback] 
                     # : список обратных вызовов Keras, применяемых во 
                     # время обучения и проверки.
                     # может быть tf.keras.callbacks.EarlyStopping

    validation_split=0.2, # По умолчанию 0.2. Доля данных обучения,
                          # которые будут использоваться в качестве 
                          # данных проверки.

    validation_data=None, # Явно переданный набор данных валидации
    **kwargs
)
```

Далее инициализируем и запускаем **ImageClassifier**. Рекомендуется проводить больше испытаний для более сложных наборов данных. Это всего лишь небольшая демонстрация на наборе данных MNIST, поэтому устанавливаем **max_trials равным 1**.

In [ ]:
# Initialize the image classifier.
clf = ak.ImageClassifier(
    directory  = OUTPUT_PATH,
    overwrite  = True,
    max_trials = 1)

# Feed the image classifier with training data.
clf.fit(x_train, y_train, epochs=10)

# Predict with the best model.
predicted_y = clf.predict(x_test)
print(predicted_y)

# Evaluate the best model with testing data.
print(clf.evaluate(x_test, y_test))

Trial 1 Complete [00h 02m 38s]
val_loss: 0.039189498871564865

Best val_loss So Far: 0.039189498871564865
Total elapsed time: 00h 02m 38s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.1552 - accuracy: 0.9517
Epoch 2/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0749 - accuracy: 0.9768
Epoch 3/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0587 - accuracy: 0.9813
Epoch 4/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0519 - accuracy: 0.9844
Epoch 5/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0466 - accuracy: 0.9855
Epoch 6/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0400 - accuracy: 0.9876
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0395 - accuracy: 0.9875
Epoch 8/10
1875/1875 [==============================] - 15s 8ms/step - loss: 0.0352 - accuracy: 0.9886


В AutoKeras реализованы такие модели, как **ResNet, Xception и отдельные CNN**, которые алгоритм может произвольно использовать и настраивать в любом месте синтезируемой модели.

По умолчанию AutoKeras использует **последние 20%** данных обучения в качестве валидационной выборки. Как показано в примере ниже, вы можете использовать **validation_split**, чтобы указать процент.

```python
clf.fit(
    x_train,
    y_train,
    # Split the training data and use the last 15% as validation data.
    validation_split=0.15,
    epochs=10,
)
```

Вы также можете использовать свой собственный набор проверки вместо отделения его от данных обучения, указав параметр **validation_data**.

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(
    x_train,
    y_train,
    # Use your own validation set.
    validation_data=(x_val, y_val),
    epochs=10,
)
x_val.shape

(10000, 28, 28)

In [ ]:
Сохранять полученные результаты будем следующим образом:

In [ ]:
from tensorflow.keras.models import model_from_json
from sklearn.metrics import classification_report

# экспортируем лучшую синтезированную модель
model = clf.export_model()
model_json = model.to_json()
# сохраняем в json-формате
with open(OUTPUT_PATH + '/autoML.json', 'w') as json_file:
    json_file.write(model_json)

# сохраняем коэффициенты
model.save_weights(OUTPUT_PATH + "/model.h5")
print("Model saved to the disk")

predicted = model.predict(x_val)
report = classification_report(y_val, np.argmax(predicted, 1))

p = os.path.join(os.path.dirname(OUTPUT_PATH), 'results.txt')
f = open(p, 'w')
f.write(report)
f.close()
print(report)

Model saved to the disk
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       991
           1       1.00      1.00      1.00      1064
           2       1.00      1.00      1.00       990
           3       1.00      1.00      1.00      1030
           4       1.00      0.99      1.00       983
           5       1.00      1.00      1.00       915
           6       1.00      1.00      1.00       967
           7       1.00      1.00      1.00      1090
           8       1.00      1.00      1.00      1009
           9       0.99      1.00      1.00       961

    accuracy                           1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000



Посмотрим на синтезированную модель

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 9216)              0     

## Customized Search Space

In [ ]:
OUTPUT_PATH_1 = "0_1_AutoMLArchSearchAutoModelMNIST/"

Опытные пользователи могут настроить пространство поиска, используя класс **AutoModel** вместо **ImageClassifier**. Здесь есть возможность настроить **ImageBlock** для некоторых высокоуровневых конфигураций, например, **block_type** - тип нейронной сети для поиска, выполнять ли нормализацию данных, и т.д. Вы также можете не указывать эти аргументы, тогда различные варианты будут настраиваться автоматически. 

In [ ]:
input_node = ak.ImageInput()
output_node = ak.ImageBlock(
    # Only search ResNet architectures.
    block_type="resnet",
    # Normalize the dataset.
    normalize=True,
    # Do not do data augmentation.
    augment=False,
)(input_node)
output_node = ak.ClassificationHead()(output_node)

clf = ak.AutoModel(
    directory  = OUTPUT_PATH_1,
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=1)

clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 21m 53s]
val_loss: 0.13894148170948029

Best val_loss So Far: 0.13894148170948029
Total elapsed time: 00h 21m 53s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1563/1563 [==============================] - 152s 97ms/step - loss: 0.3946 - accuracy: 0.9100
Epoch 2/10
1563/1563 [==============================] - 151s 96ms/step - loss: 0.2824 - accuracy: 0.9433
Epoch 3/10
1563/1563 [==============================] - 151s 97ms/step - loss: 0.2243 - accuracy: 0.9533
Epoch 4/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.1751 - accuracy: 0.9627
Epoch 5/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.1278 - accuracy: 0.9692
Epoch 6/10
1563/1563 [==============================] - 150s 96ms/step - loss: 0.1533 - accuracy: 0.9647
Epoch 7/10
1563/1563 [==============================] - 151s 97ms/step - loss: 0.1125 - accuracy: 0.9719
Epoch 8/10
1563/1563 [==============================] - 151s 97ms/step - loss: 0.1579 - acc

In [ ]:
# экспортируем лучшую синтезированную модель
model = clf.export_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 28, 28, 1)    3           input_1[0][0]                    
__________________________________________________________________________________________________
resizing (Resizing)             (None, 32, 32, 1)    0           normalization[0][0]              
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 32, 32, 3)    0           resizing[0][0]                   
                                                                 resizing[0][0]               

Использование AutoModel аналогично функциональному API Keras. По сути, вы строите граф, ребра которого являются блоками, а узлы - промежуточными выходами блоков. **Чтобы добавить ребро из input_node в output_node необходимо сделать следующее: output_node = ak.some_block(input_node)**.

Вы также можете использовать более мелкие блоки для дальнейшей настройки пространства поиска.

In [ ]:
OUTPUT_PATH_2 = "0_2_AutoMLArchSearchAutoModelMNIST/"

input_node = ak.ImageInput()
output_node = ak.Normalization()(input_node)
output_node = ak.ImageAugmentation(horizontal_flip=False)(output_node)
output_node = ak.ResNetBlock(version="v2")(output_node)
output_node = ak.ClassificationHead()(output_node)
clf = ak.AutoModel(
    directory  = OUTPUT_PATH_2,
    inputs=input_node,
    outputs=output_node,
    overwrite=True,
    max_trials=1)
clf.fit(x_train, y_train, epochs=10)

Trial 1 Complete [00h 08m 01s]
val_loss: 0.7793896198272705

Best val_loss So Far: 0.7793896198272705
Total elapsed time: 00h 08m 01s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
1875/1875 [==============================] - 45s 24ms/step - loss: 1.1632 - accuracy: 0.6194
Epoch 2/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.9478 - accuracy: 0.6888
Epoch 3/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.9088 - accuracy: 0.7007
Epoch 4/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.8818 - accuracy: 0.7087
Epoch 5/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.8756 - accuracy: 0.7114
Epoch 6/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.8643 - accuracy: 0.7149
Epoch 7/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.8506 - accuracy: 0.7175
Epoch 8/10
1875/1875 [==============================] - 45s 24ms/step - loss: 0.8511 - accuracy: 0.7

Здесь важно отметить, что блоки не являются последовательными, даже если они выглядят так, как в коде. Их можно рассматривать как дополнительные сервисы и указатели для AutoModel в каждой строке.

In [ ]:
# экспортируем лучшую синтезированную модель
model = clf.export_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
normalization (Normalization)   (None, 28, 28, 1)    3           input_1[0][0]                    
__________________________________________________________________________________________________
random_flip (RandomFlip)        (None, 28, 28, 1)    0           normalization[0][0]              
__________________________________________________________________________________________________
resizing (Resizing)             (None, 32, 32, 1)    0           random_flip[0][0]                
______________________________________________________________________________________________